In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#extract dataset
import zipfile
import os

#MSI_MSS
ZIP_PATH = "/content/gdrive/MyDrive/MSI_MSS_zips/Train/MSI.zip"
IMAGE_FOLDER = "/content/gdrive/MyDrive/MSI_MSS/Train/"

# Ensure that the extraction folder exists
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

# Open the zip file and extract its contents without creating a new folder
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        print(f"Extracting: {file_info.filename}")
        zip_ref.extract(file_info, path=IMAGE_FOLDER)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
from sklearn.ensemble import IsolationForest
from joblib import dump

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load dataset
data_dir = '/content/gdrive/MyDrive/MSI_MSS/Train'

In [ ]:
#preprocess data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Feature extraction model (ResNet50)
feature_extractor = models.resnet50(pretrained=True)
feature_extractor.fc = torch.nn.Sequential()  # Remove the final layer
feature_extractor.to(device)
feature_extractor.eval()

# Extract features
features = []
for inputs, _ in dataloader:
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = feature_extractor(inputs)
        features.append(outputs.cpu().numpy())
features = np.concatenate(features, axis=0)

# Save the feature extractor model
feature_extractor_path = '/content/gdrive/MyDrive/feature_extractor.pth'
torch.save(feature_extractor.state_dict(), feature_extractor_path)

In [ ]:
# Train Isolation Forest
clf = IsolationForest(random_state=0, contamination='auto')
clf.fit(features)

# Save the trained model
isolation_forest_model_path = '/content/gdrive/MyDrive/Anomaly_MSI_MSS_Isolation_Forest_model.joblib'
dump(clf, isolation_forest_model_path)

['/content/gdrive/MyDrive/Anomaly_MSI_MSS_Isolation_Forest_model.joblib']

In [ ]:
!ls /content/gdrive/MyDrive/MSI_MSS_2/Test_Final/MSI/MSI

In [ ]:
from joblib import load
from PIL import Image

# Load the trained Isolation Forest model
isolation_forest_model_path = '/content/gdrive/MyDrive/Anomaly_MSI_MSS_Isolation_Forest_model.joblib'
clf = load(isolation_forest_model_path)

# Anomaly detection function
def is_anomaly(image_path, clf, feature_extractor, transform):
    image = Image.open(image_path).convert('RGB')
    input_image = transform(image).unsqueeze(0).to(device)

    feature_extractor.to(device)
    with torch.no_grad():
        image_features = feature_extractor(input_image)

    is_outlier = clf.predict(image_features.cpu().numpy().reshape(1, -1))
    return is_outlier[0] == -1

feature_extractor_path = '/content/gdrive/MyDrive/feature_extractor.pth'
# Define the model architecture
feature_extractor = models.resnet50(pretrained=False)  # pretrained=False since loading custom weights
feature_extractor.fc = torch.nn.Sequential()  # Ensure this matches the saved model's structure

# Load the saved weights
feature_extractor.load_state_dict(torch.load(feature_extractor_path))

# Move to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)
feature_extractor.eval()

# Example usage
image_path = '/content/gdrive/MyDrive/Cow.png'
if is_anomaly(image_path, clf, feature_extractor, transform):
    print("Anomaly detected. Image will not be classified.")
else:
    print("No anomaly detected. Proceeding with classification.")
    # Load and use your MSI_MSS classification model here


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Anomaly detected. Image will not be classified.
